<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Structure-cheat-sheet" data-toc-modified-id="Structure-cheat-sheet-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Structure cheat sheet</a></span></li><li><span><a href="#Data-structure" data-toc-modified-id="Data-structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data structure</a></span></li><li><span><a href="#get-features" data-toc-modified-id="get-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>get features</a></span></li><li><span><a href="#Exploration-of-the-Features" data-toc-modified-id="Exploration-of-the-Features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exploration of the Features</a></span></li><li><span><a href="#Exploring-spectral-features" data-toc-modified-id="Exploring-spectral-features-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Exploring spectral features</a></span></li></ul></div>

# Basics

## Structure cheat sheet

1. func: train data lead (following order)
    1. read the descriptive dataframe from the feature-pipeline
    2. extract feature from the feature-objects which are labeled train-dataset from dataframe
    3. create numpy feature array for the processing pipeline
2. preprocessing
    1. Transformation (any combination of the following)
        + log-transform
        + PCA
        + others
    2. Scaling (one of the following)
        + StandardScaler
        + MinMaxScaler
3. Unsupervised Clustering
    1. Estimate initial hyperparameter
    2. Create grid over various hyperparameters
    3. Train all and choose the best according to metric
    
    
in all steps the cluster-recorder object (possibly dataframe-row) will record all the meta-information like hyper-parameters

## Data structure

There are multiple degrees of freedom in the data:

1. Signal to noise ratio (SNR)
2. Machine type
    1. pump
    2. fan
    3. valve (solenoid)
    4. slider
3. Machine ID
    1. four different machine IDs
    
The pipeline will be applied to fixed SNR, fixed machine type and fixed ID

## get features

Get the descriptive dataframe for the features.

The descriptive dataframe contains all IDs of the pump. We will focus on ID '00' for now since the modeling phase is seperated per SNR, per machine, per ID anyway.

class: 
+ uni\_\<model\>
attributes:
+ default threshold
+ roc_auc
methods:
+ fit
+ predict
+ predict_score
+ eval_roc_auc

In [10]:
#===============================================
# Basic Imports
BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from tqdm import tqdm
import glob


load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractor_ICA2
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother
load Simple_FIR_HP
load TimeSliceAppendActivation
load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load tensorflow models
Load detection_pipe


The exploration of the Dimensionality Reduction was already done in another notebook. We derived the following rules of thumb:

1. PCA and ICA deliver almost the same results looking at the relative absolute error
2. PCA is usually much faster
3. on PSD and the ICA demix matrix, no dimensionality reduction is needed
4. for a framed Mel-spectrum a number of components between 32 and 64 is a good measure. The resulting error is about 2-4%
5. for a whole Mel-spectrum a number of components between 64 and 128 is advised

## Exploration of the Features

In [4]:
diagram = 'extdia_v1'
machines = ['pump'
            , 'fan'
            , 'slider'
            , 'valve'
            ]
SNRs = ['6dB', 'min6dB'
        ]
IDs = ['00'
       , '02'
       ,# '04'
       #, '06'
        ]
features = [#('MEL_denbssm', {'function':'frame', 'frames':3})
            #, ('MEL_denbssm', {'function':'frame', 'frames':5})
            #, ('MEL_denbssm', {'function':'frame', 'frames':7})
            #, ('MEL_denbssm', {'function':'flat'})
            ('PSD_denbssm', {'function':'channel'})
            #, ('MEL_bssm', {'function':'frame', 'frames':3})
            #, ('MEL_bssm', {'function':'frame', 'frames':5})
            #, ('MEL_bssm', {'function':'frame', 'frames':7})
            #, ('MEL_bssm', {'function':'flat'})
            , ('PSD_bssm', {'function':'channel'})
            #, ('MEL_raw', {'function':'frame', 'frames':3})
            #, ('MEL_raw', {'function':'frame', 'frames':5})
            #, ('MEL_raw', {'function':'frame', 'frames':7})
            #, ('MEL_raw', {'function':'flat'})
            , ('PSD_raw', {'function':'channel'})
            #, ('MEL_den', {'function':'frame', 'frames':3})
            #, ('MEL_den', {'function':'frame', 'frames':5})
            #, ('MEL_den', {'function':'frame', 'frames':7})
            #, ('MEL_den', {'function':'flat'})
            , ('PSD_den', {'function':'channel'})
            , ('ICA_demix', {'function':'flat'})
            , ('ICA_demix', {'function':'maxrange'})
            ]

tasks = [{
        'path_descr':glob.glob(BASE_FOLDER 
                               + '/dataset/{}/{}{}{}_EDiaV1'.format(diagram, machine, SNR, ID) 
                               + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    #(PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {
    'n_estimators':100
    ,'max_features':1
    ,'random_state':42})

# modeling = (uni_EllipticEnvelope, {})

pipes = [Pipe(preprocessing, modeling) for i in range(len(tasks))]

tasks_failed = []
for pipe, task in tqdm(zip(pipes, tasks), total=len(tasks)):
    try:
        pipe.run_pipe(task)
    except:
        tasks_failed.append(task)
        print('Task failed')

  0%|                                                                                           | 0/96 [00:00<?, ?it/s]

...loading data
../..//dataset/extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model


  1%|▊                                                                                  | 1/96 [00:00<01:16,  1.25it/s]

model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.4743997261479779
pipe saved to pickle
...loading data
../..//dataset/extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc:

  2%|█▋                                                                                 | 2/96 [00:01<01:16,  1.23it/s]

...

 94%|████████████████████████████████████████████████████████████████████████████▉     | 90/96 [00:50<00:00,  8.69it/s]

Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB02_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed

 97%|███████████████████████████████████████████████████████████████████████████████▍  | 93/96 [00:50<00:00, 10.77it/s]


...loading data
../..//dataset/extdia_v1\valvemin6dB02_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB02_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB02_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB02_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed
...loading data
../..//dataset/extdia_v1\valvemin6dB02_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
Task failed

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [00:50<00:00,  1.89it/s]

## Exploring spectral features

In [9]:
diagram = 'extdia_v1'
machines = ['pump'
            , 'fan'
            , 'slider'
            , 'valve'
            ]
SNRs = ['6dB', 'min6dB'
        ]
IDs = ['00'
       , '02'
       ,# '04'
       #, '06'
        ]
features = [('MEL_denbssm', {'function':'frame', 'frames':3})
            , ('MEL_denbssm', {'function':'frame', 'frames':5})
            , ('MEL_denbssm', {'function':'frame', 'frames':7})
            , ('MEL_denbssm', {'function':'flat'})
            #('PSD_denbssm', {'function':'channel'})
            , ('MEL_bssm', {'function':'frame', 'frames':3})
            , ('MEL_bssm', {'function':'frame', 'frames':5})
            , ('MEL_bssm', {'function':'frame', 'frames':7})
            , ('MEL_bssm', {'function':'flat'})
            #, ('PSD_bssm', {'function':'channel'})
            , ('MEL_raw', {'function':'frame', 'frames':3})
            , ('MEL_raw', {'function':'frame', 'frames':5})
            , ('MEL_raw', {'function':'frame', 'frames':7})
            , ('MEL_raw', {'function':'flat'})
            #, ('PSD_raw', {'function':'channel'})
            , ('MEL_den', {'function':'frame', 'frames':3})
            , ('MEL_den', {'function':'frame', 'frames':5})
            , ('MEL_den', {'function':'frame', 'frames':7})
            , ('MEL_den', {'function':'flat'})
            #, ('PSD_den', {'function':'channel'})
            #, ('ICA_demix', {'function':'flat'})
            #, ('ICA_demix', {'function':'maxrange'})
            ]

tasks = [{
        'path_descr':glob.glob(BASE_FOLDER 
                               + '/dataset/{}/{}{}{}_EDiaV1'.format(diagram, machine, SNR, ID) 
                               + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {
    'n_estimators':64
    ,'max_features':4
    ,'random_state':42
    ,'n_jobs':1
})

pipes = [Pipe(preprocessing, modeling) for i in range(len(tasks))]

# create the threads
n_jobs = 2
worker_list = []
queue = Queue()
for worker in range(n_jobs):
    worker = PipeThread(queue)
    worker.daemon = True
    worker.start()
    worker_list.append(worker)

In [10]:
diagram = 'extdia_v1'
machines = ['pump'
            , 'fan'
            , 'slider'
            , 'valve'
            ]
SNRs = ['6dB', 'min6dB'
        ]
IDs = ['00'
       , '02'
       ,# '04'
       #, '06'
        ]
features = [('MEL_denbssm', {'function':'frame', 'frames':3})
            , ('MEL_denbssm', {'function':'frame', 'frames':5})
            , ('MEL_denbssm', {'function':'frame', 'frames':7})
            , ('MEL_denbssm', {'function':'flat'})
            #('PSD_denbssm', {'function':'channel'})
            , ('MEL_bssm', {'function':'frame', 'frames':3})
            , ('MEL_bssm', {'function':'frame', 'frames':5})
            , ('MEL_bssm', {'function':'frame', 'frames':7})
            , ('MEL_bssm', {'function':'flat'})
            #, ('PSD_bssm', {'function':'channel'})
            , ('MEL_raw', {'function':'frame', 'frames':3})
            , ('MEL_raw', {'function':'frame', 'frames':5})
            , ('MEL_raw', {'function':'frame', 'frames':7})
            , ('MEL_raw', {'function':'flat'})
            #, ('PSD_raw', {'function':'channel'})
            , ('MEL_den', {'function':'frame', 'frames':3})
            , ('MEL_den', {'function':'frame', 'frames':5})
            , ('MEL_den', {'function':'frame', 'frames':7})
            , ('MEL_den', {'function':'flat'})
            #, ('PSD_den', {'function':'channel'})
            #, ('ICA_demix', {'function':'flat'})
            #, ('ICA_demix', {'function':'maxrange'})
            ]

tasks = [{
        'path_descr':glob.glob(BASE_FOLDER 
                               + '/dataset/{}/{}{}{}_EDiaV1'.format(diagram, machine, SNR, ID) 
                               + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {
    'n_estimators':64
    ,'max_features':4
    ,'random_state':42
    ,'n_jobs':1
})

# modeling = (uni_EllipticEnvelope, {})

pipes = [Pipe(preprocessing, modeling) for i in range(len(tasks))]


# create the threads
n_jobs = 2
worker_list = []
queue = Queue()
for worker in range(n_jobs):
    worker = PipeThread(queue)
    worker.daemon = True
    worker.start()
    worker_list.append(worker)

# fill the Queue
task_status = []
for pipe, task in (zip(pipes, tasks)):
    task_status.append(queue.put((pipe, task)))

../..//dataset/extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
../..//dataset/extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.8564387632896486
pipe saved to pickle
../..//dataset/extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
evaluation successfull, roc_auc: 0.8516784366237957
pipe saved to pickle
../..//dataset/extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc

evaluation successfull, roc_auc: 0.612769120060396
pipe saved to pickle
../..//dataset/extdia_v1\pumpmin6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.8203824147880093
pipe saved to pickle
../..//dataset/extdia_v1\pumpmin6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
evaluation successfull, roc_auc: 0.6824297559606876
pipe saved to pickle
../..//dataset/extdia_v1\pumpmin6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.6402769184057849
pipe saved to pickle
../..//dat

data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.6695033581650873
pipe saved to pickle
../..//dataset/extdia_v1\fan6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.7353983422779492
pipe saved to pickle
../..//dataset/extdia_v1\fan6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
evaluation successfull, roc_auc: 0.6795468351774204
pipe saved to pickle
../..//dataset/extdia_v1\fan6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data preprocessing finished

...fitting the model
data loading completed

...preprocessing data
model fitt

model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.5600660182953127
pipe saved to pickle
../..//dataset/extdia_v1\fanmin6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
evaluation successfull, roc_auc: 0.5557621348664279
pipe saved to pickle
../..//dataset/extdia_v1\fanmin6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.5748238745781743
pipe saved to pickle
../..//dataset/extdia_v1\fanmin6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data preprocessing finished

...fitting the model
data loading completed

...preprocessing data
model fitted successfully

...evaluating model
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.526341812870338

pipe saved to pickle
../..//dataset/extdia_v1\fanmin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
evaluation successfull, roc_auc: 0.7334367201488862
pipe saved to pickle
../..//dataset/extdia_v1\fanmin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.7622164652952521
pipe saved to pickle
../..//dataset/extdia_v1\fanmin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.7377192914393899
pipe saved to pickle
../..//dataset/extdia_v1\slider6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...load

data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.925773503543081
pipe saved to pickle
../..//dataset/extdia_v1\slider6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.9051844514300773
pipe saved to pickle
../..//dataset/extdia_v1\slider6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.9118436710635416
pipe saved to pickle
../..//dataset/extdia_v1\slider6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
mo

data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.8053977471980249
pipe saved to pickle
../..//dataset/extdia_v1\slidermin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.7264619721202117
pipe saved to pickle
../..//dataset/extdia_v1\slidermin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.7534697318053691
pipe saved to pickle
../..//dataset/extdia_v1\slidermin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluati

../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB00_EDiaV1HPaug0TsSl_pandaDisc.pkl --> Done
...loading data
Something went wrong
../..//dataset/extdia_v1\valvemin6dB02_EDiaV1HPaug0TsSl_pandaDisc.pkl --

In [11]:
diagram = 'extdia_v1'
machines = ['pump'
            , 'fan'
            , 'slider'
            , 'valve'
            ]
SNRs = ['6dB', 'min6dB'
        ]
IDs = ['00'
       , '02'
       ,# '04'
       #, '06'
        ]
features = [('MEL_denbssm', {'function':'frame', 'frames':3})
            , ('MEL_denbssm', {'function':'frame', 'frames':5})
            , ('MEL_denbssm', {'function':'frame', 'frames':7})
            , ('MEL_denbssm', {'function':'flat'})
            #('PSD_denbssm', {'function':'channel'})
            , ('MEL_bssm', {'function':'frame', 'frames':3})
            , ('MEL_bssm', {'function':'frame', 'frames':5})
            , ('MEL_bssm', {'function':'frame', 'frames':7})
            , ('MEL_bssm', {'function':'flat'})
            #, ('PSD_bssm', {'function':'channel'})
            , ('MEL_raw', {'function':'frame', 'frames':3})
            , ('MEL_raw', {'function':'frame', 'frames':5})
            , ('MEL_raw', {'function':'frame', 'frames':7})
            , ('MEL_raw', {'function':'flat'})
            #, ('PSD_raw', {'function':'channel'})
            , ('MEL_den', {'function':'frame', 'frames':3})
            , ('MEL_den', {'function':'frame', 'frames':5})
            , ('MEL_den', {'function':'frame', 'frames':7})
            , ('MEL_den', {'function':'flat'})
            #, ('PSD_den', {'function':'channel'})
            #, ('ICA_demix', {'function':'flat'})
            #, ('ICA_demix', {'function':'maxrange'})
            ]

tasks = [{
        'path_descr':glob.glob(BASE_FOLDER 
                               + '/dataset/{}/{}{}{}_EDiaV1'.format(diagram, machine, SNR, ID) 
                               + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_AutoEncoder, {})

# modeling = (uni_EllipticEnvelope, {})

pipes = [Pipe(preprocessing, modeling) for i in range(len(tasks))]


# create the threads
n_jobs = 1
worker_list = []
queue = Queue()
for worker in range(n_jobs):
    worker = PipeThread(queue)
    worker.daemon = True
    worker.start()
    worker_list.append(worker)

In [12]:
pipes[0].run_pipe(tasks[0])

../..//dataset/extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
Epoch 1/50
1049/1049 [==============================] - 13s 12ms/step - loss: 0.9346
Epoch 2/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.8727
Epoch 3/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.8454
Epoch 4/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.8268
Epoch 5/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.8138
Epoch 6/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.8028
Epoch 7/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.7939
Epoch 8/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.7853
Epoch 9/50
1049/1049 [==============================] - 8s 8ms/step - loss: 0.7792
Epoch 10/50
1049/1049 [==============================] - 8s 8ms/step

ValueError: Wrong number of items passed 64, placement implies 1

In [31]:
data_train, data_test = pipes[0].get_data()
data_train, data_test = pipes[0].preprocess(data_train, data_test)

In [21]:
data_test.shape

(88946, 64)

In [25]:
pipes[0].model.predict_score(data_test).shape

(88946,)

In [30]:
pipes[0].ground_truth.shape

(88946,)

In [32]:
pipes[0].model.eval_roc_auc(data_test, pipes[0].ground_truth)

0.06716138156255405

In [28]:
pipes[0].evaluate(data_test, pipes[0].ground_truth)

ValueError: Wrong number of items passed 64, placement implies 1

In [35]:
pipes[0].model.super()

TypeError: super() argument 1 must be type, not uni_AutoEncoder

In [ ]:
# fill the Queue
task_status = []
for pipe, task in (zip(pipes[1:5], tasks[1:5])):
    queue.put((pipe, task))